# Test Company Analysis Report

증권사의 기업 리포트 데이터셋에 대해해 nDCG, Precision, Recall과 같은 평가 지표를 통해 벡터 검색 테스트를 수행합니다.

In [1]:
import os, json
from dotenv import load_dotenv
import time 
import joblib
import requests
from elasticsearch import Elasticsearch
import pandas as pd
from openai import OpenAI # openai==1.52.2
import traceback
import ast

from sentence_transformers import SentenceTransformer

from requests.exceptions import RequestException

from tqdm.auto import tqdm

In [2]:

load_dotenv()
FAISS_API_KEY = os.getenv("API_KEY")
print(FAISS_API_KEY) ## OPEN_API_KEY 속성이 존재해야함

nlp13_access_token_IJkdk3d2


In [3]:
# 청크 데이터 불러오기
document_data = joblib.load('../data/final_documents_ver2.pkl')
# 테스트 데이터 불러오기

In [4]:
print(document_data[0])
print(document_data[0].metadata['nid'])

page_content='20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.

Brief Description: Equity Research
2024.5.9

| 투자의견(유지) | 매수 |
| --- | --- |
| 목표주가(상향) ▲ | 370,000원 |
| 현재주가(24/5/8) | 260,000원 |
| 상승여력 | 42.3% |
| 영 업이익(24F,십억원) | 801 |
| Consensus 영업이익(24F,십억원) | 777 |
| EPS 성장률(24F,%) | 25.1 |
| MKT EPS 성장률(24F,%) | 75.9 |
| P/E(24F,x) | 17.0 |
| MKT P/E(24F,x) | 11.1 |
| KOSPI | 2,745.05 |
| 시가총액(십억원) | 12,575 |
| 발행주식수(백만주) | 48 |
| 유동주식비율(%) | 58.8 |
| 외국인 보유비중(%) | 36.5 |
| 베타(12M) 일간수익률 | 0.66 |
| 52주 최저가(원) | 146,500 |
| 52주 최고가(원) | 260,000 |
| (%) 1M 절대주가 6.8 | 6M 12M 36.3 28.1 |
| 상대주가 |  |
| 5.7 | 20.2 17.3 |
| 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 | 140 크래프톤 KOSPI 130 120 110 100 90 80 70 60 23.4 23.8 23.12 24.4 |
' metadata={'category': 'table', 'coordinates': [{'x': 0.041, 'y': 0.2669}, {'x': 0.2923, 'y': 0.2669}, {'x': 0.2923, 'y': 0.7877}, {'x': 0.041, 'y': 0.7877}], 'page': 1, 'id': 1, 'company_name': '크래프톤', 'report_date': '20240509', 'securitie

In [5]:
    vector_text = str(document_data[0].metadata['summary']) + '\n\n' + document_data[0].page_content + '\n\n' + str(document_data[0].metadata['expected_questions'])
    print(vector_text)

2024년 5월 9일 미래에셋증권의 크래프톤 관련 레포트 요약:

- **투자의견**: 유지
- **매수**: 목표주가 370,000원 (상향)
- **현재주가**: 260,000원 (2024년 5월 8일 기준)
- **상승여력**: 42.3%
- **영업이익(2024F)**: 801십억원 (Consensus: 777십억원)
- **EPS 성장률(2024F)**: 25.1%
- **MKT EPS 성장률(2024F)**: 75.9%
- **P/E(2024F)**: 17.0x (MKT P/E: 11.1x)
- **KOSPI**: 2,745.05
- **시가총액**: 12,575십억원
- **발행주식수**: 48백만주
- **유동주식비율**: 58.8%
- **외국인 보유비중**: 36.5%
- **베타(12M)**: 0.66
- **52주 최저가**: 146,500원
- **52주 최고가**: 260,000원
- **1M 절대주가**: 6.8%, 6M: 36.3%, 12M: 28.1%
- **상대주가**: 5.7%, 20.2%, 17.3%

이 데이터는 크래프톤의 투자 가치를 평가하는 데 중요한 지표로 활용될 수 있습니다.

20240509에 미래에셋증권에서 발행한 크래프톤에 관한 레포트에서 나온 내용.

Brief Description: Equity Research
2024.5.9

| 투자의견(유지) | 매수 |
| --- | --- |
| 목표주가(상향) ▲ | 370,000원 |
| 현재주가(24/5/8) | 260,000원 |
| 상승여력 | 42.3% |
| 영 업이익(24F,십억원) | 801 |
| Consensus 영업이익(24F,십억원) | 777 |
| EPS 성장률(24F,%) | 25.1 |
| MKT EPS 성장률(24F,%) | 75.9 |
| P/E(24F,x) | 17.0 |
| MKT P/E(24F,x) | 11.1 |
| KOSPI | 2,745.05 |
| 시가총액(십억원) | 12,575 |
| 발행주식수(백만주) | 48 |

## 점수 구조화

각 테스트 문항에 대해 점수를 스케일링 하기 위한 구조를 짭니다.

각 그룹 별로 리스트를 구조화 합니다.

In [6]:
company_group = {}
source_group = {}
for chunk in document_data :
  metadata = chunk.metadata
  
  if company_group.get(metadata['company_name']) is None :
    company_group[metadata['company_name']] = [metadata['nid']]
  else :
    company_group[metadata['company_name']].append(metadata['nid'])
    
  if source_group.get(metadata['source_file']) is None :
    source_group[metadata['source_file']] = [metadata['nid']]
  else :
    source_group[metadata['source_file']].append(metadata['nid'])

In [7]:
print(company_group.keys())


dict_keys(['크래프톤', 'CJ제일제당', '엘앤에프', 'SK하이닉스', 'SK케미칼', '카카오뱅크', '한화솔루션', '롯데렌탈', 'LG화학', '네이버'])


In [8]:
print(company_group)

{'크래프톤': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 2398, 2399, 2400, 2401, 2402, 2403, 2404, 2405, 2406, 2407, 2408, 2409, 2410, 2411, 2412, 2413, 2414, 2415, 2416, 2417, 2418, 2419, 2420, 2527, 2528, 2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539, 2540, 2541, 2542, 2543, 2544, 2545, 2546, 2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554

In [9]:
print(source_group.keys())

dict_keys(['크래프톤_20240509_미래에셋증권.pdf', 'CJ제일제당_20241120_신한투자증권.pdf', '크래프톤_20241017_교보증권.pdf', '엘앤에프_20240807_IM증권.pdf', 'SK하이닉스_20241025_DS증권.pdf', 'SK케미칼_20240813_SK증권.pdf', 'SK하이닉스_20241209_유진투자증권.pdf', '카카오뱅크_20241127_삼성증권.pdf', '크래프톤_20241010_하나증권.pdf', '한화솔루션_20241031_유안타증권.pdf', '엘앤에프_20241017_삼성증권.pdf', '한화솔루션_20241030_삼성증권.pdf', '롯데렌탈_20241106_신한투자증권.pdf', '크래프톤_20241224_Daishin증권.pdf', '롯데렌탈_20241213_교보증권.pdf', 'CJ제일제당_20241113_키움증권.pdf', 'CJ제일제당_20241113_IM증권.pdf', 'CJ제일제당_20241113_교보증권.pdf', '롯데렌탈_20241106_삼성증권.pdf', 'LG화학_20241029_신한투자증권.pdf', '한화솔루션_20241031_미래에셋증권.pdf', 'SK케미칼_20230816_한화투자증권.pdf', '카카오뱅크_20240508_한화투자증권.pdf', '엘앤에프_20241104_DS투자증권.pdf', 'LG화학_20241029_IM증권.pdf', 'LG화학_20241029_DaiShin증권.pdf', '한화솔루션_20241031_DS투자증권.pdf', 'CJ제일제당_20241113_IBK투자증권.pdf', 'SK케미칼_20240208_DB금융투자.pdf', 'CJ제일제당_20241024_DS투자증권.pdf', '한화솔루션_20241031_IBK투자증권.pdf', '네이버_20241108_IBK투자증권.pdf', 'LG화학_20241029_SK증권.pdf', '카카오뱅크_20240509_NH투자증권.pdf', 'SK하이닉스_20241122_키움증권.pdf', '엘앤에프

In [10]:
print(source_group)

{'크래프톤_20240509_미래에셋증권.pdf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], 'CJ제일제당_20241120_신한투자증권.pdf': [29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92], '크래프톤_20241017_교보증권.pdf': [93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128], '엘앤에프_20240807_IM증권.pdf': [129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167], 'SK하이닉스_20241025_DS증권.pdf': [168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 

## 테스트 실행

In [11]:
qa_pairs = pd.read_csv('../data/report_qa_pairs_20250205.csv')

In [12]:
# 1. 텍스트 임베딩 모델 로드 (Hugging Face SentenceTransformer)
model = SentenceTransformer('dragonkue/BGE-m3-ko')  # 예

In [13]:
# faiss 클라이언트 설정
session = requests.Session()
url = "http://3.34.62.202:8060/"

In [14]:
# es 클라이언트 설정
es = Elasticsearch("http://3.34.62.202:9200")

In [30]:
es_index_name = "prod-labq-documents-final-v2-bge-m3-ko-v1-4"
index_name = "prod-labq-documents-final-v2-bge-m3-ko-v1-4"

In [31]:
# 2. 텍스트 데이터를 벡터로 변환
text = "Faiss is a distributed, RESTful search engine."
vector = model.encode(text).tolist()  # 벡터를 리스트로 변환
print("vector", vector)
dimension = len(vector)
print("dimension:", dimension)

vector [-0.023218566551804543, -0.034254290163517, -0.030382176861166954, 0.021542740985751152, -0.03141197934746742, 0.001701395958662033, 0.044147323817014694, 0.019730012863874435, -0.018713099882006645, 0.05151927098631859, -0.03658164292573929, -0.0020344264339655638, -0.002867817645892501, 0.021956101059913635, 0.01835559494793415, 0.01578981988132, 0.008035476319491863, -0.025734996423125267, -0.01861422508955002, -0.02501322515308857, -0.03177196905016899, 0.05051146447658539, 0.021209388971328735, -0.002957786899060011, 0.05762643367052078, 0.03805505484342575, -0.0243743434548378, -0.03242011368274689, 0.008425990119576454, -0.029842093586921692, 0.03277371823787689, 0.02034126967191696, 0.03448186814785004, -0.017877643927931786, -0.04802555590867996, -0.033362630754709244, -0.056233812123537064, -0.02778574824333191, -0.04041918367147446, -0.017456896603107452, -0.019937938079237938, 0.019160909578204155, 0.013187439180910587, -0.02175966091454029, 0.025715388357639313, -0.

In [32]:
# 3. faiss 인덱스 생성
response = session.post(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "algorithm" : 1, "dimension": dimension}, timeout=15)
print(response.json())

{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-final-v2-bge-m3-ko-v1-4'}


In [33]:
# 4. faiss에 인덱스가 존재하는 지 확인
response = session.get(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, params={"index" : index_name}, timeout=15)
print(response.json())

{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-final-v2-bge-m3-ko-v1-4'}


In [34]:
# 5. Elasticsearch에 인덱스 존재하는 지 확인

if not es.indices.exists(index=es_index_name):
    response = es.indices.create(index=es_index_name)
    print(response)
    print("Elasticsearch 인덱스 생성 완료")
    response = es.indices.refresh(index=es_index_name)
    print(response)
else:
    print("Elasticsearch 인덱스 이미 존재")
response = es.indices.stats(index=es_index_name)
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'prod-labq-documents-final-v2-bge-m3-ko-v1-4'}
Elasticsearch 인덱스 생성 완료
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 0, 'deleted': 0, 'total_size_in_bytes': 0}, 'shard_stats': {'total_count': 1}, 'store': {'size_in_bytes': 227, 'total_data_set_size_in_bytes': 227, 'reserved_in_bytes': 0}, 'indexing': {'index_total': 0, 'index_time_in_millis': 0, 'index_current': 0, 'index_failed': 0, 'delete_total': 0, 'delete_time_in_millis': 0, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0, 'write_load': 0.0}, 'get': {'total': 0, 'time_in_millis': 0, 'exists_total': 0, 'exists_time_in_millis': 0, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 0, 'query_time_in_millis': 0, 'query_current': 0, 'query_failure': 0, 'fetch_total

In [ ]:
# 5. 데이터 대량 삽입

max_retries = 3
for chunk in tqdm(document_data, desc="Processing document chunks", leave=False):
    # print(chunk.page_content)
    # print(chunk.metadata['nid'])
    # print(chunk.metadata['summary'])
    # print(chunk.metadata['expected_questions'])
    nid = chunk.metadata['nid'] + 1
    uid = chunk.metadata['uuid']
    text = chunk.page_content
    metadata = chunk.metadata
    vector_text = ''
    if metadata.get('summary') :
        vector_text += str(metadata['summary']) + '\n\n'
        
    vector_text += text + '\n\n' + str(metadata['expected_questions'])
    vector = model.encode(vector_text).tolist()  # 벡터를 리스트로 변환
    for attempt in range(1, max_retries + 1) :
        try :
            response = session.post(f"{url}/api/context", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "input_vector" : vector}, timeout=15)
            response.raise_for_status()
            
            break
        except RequestException as e:
            tqdm.write(f"Attempt {attempt} failed: {e}")
            # 마지막 시도라면 예외를 다시 발생시킵니다.
            if attempt == max_retries:
                raise
            # 재시도하기 전에 약간 대기 (예: 2초)
            time.sleep(2)

    
    tqdm.write(f"{response.json()}")
    

    # Elasticsearch에 데이터 삽입
    doc = {
        "uid": uid,
        "nid": nid,
        "text": text,
        "embedding": vector,
        "total_text": vector_text,
        "metadata": metadata
        
    }
    response = es.index(index=es_index_name, body=doc)
    tqdm.write(f"Document indexed: {response}")

In [46]:
print(len(document_data))

3629


In [18]:
# 6. 데이터 조회
query_text = "CJ제일제당의 2025년 매출액은 얼마로 예상되나요?"
query_vector = model.encode(query_text).tolist()

response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 10}, timeout=15)
print(response.json())

{'status': 'ok', 'results': [{'document_id': 814, 'distance': 0.594671905040741}, {'document_id': 1116, 'distance': 0.6060703992843628}, {'document_id': 3225, 'distance': 0.6082162857055664}, {'document_id': 811, 'distance': 0.6195539832115173}, {'document_id': 1170, 'distance': 0.6273811459541321}, {'document_id': 1180, 'distance': 0.6300528049468994}, {'document_id': 3228, 'distance': 0.6471505165100098}, {'document_id': 3257, 'distance': 0.6556746959686279}, {'document_id': 36, 'distance': 0.6562689542770386}, {'document_id': 3268, 'distance': 0.6626720428466797}]}


In [19]:
# 8. 반환된 데이터를 디코딩
faiss_results = response.json()['results']
for item in faiss_results:
  print(item)
  print(document_data[item['document_id']])
  print("")

{'document_id': 814, 'distance': 0.594671905040741}
page_content='20241113에 교보증권에서 발행한 CJ제일제당에 관한 레포트에서 나온 내용.

| 12 결산(십억원) | 2022A | 2023A | 2024F | 2025F | 2026F |
| --- | --- | --- | --- | --- | --- |
| 매출액 | 30,080 | 29,023 | 29,563 | 30,707 | 32,242 |
| 매출원가 | 23,525 | 22,971 | 23,119 | 24,013 | 25,213 |
| 매출총이익 | 6,555 | 6,053 | 6,445 | 6,694 | 7,029 |
| 매출총이익률 (%) | 21.8 | 20.9 | 21.8 | 21.8 | 21.8 |
| 판매비와관리비 | 4,890 | 4,761 | 4,876 | 5,034 | 5,034 |
| 영업이익 | 1,665 | 1,292 | 1,569 | 1,660 | 1,902 |
| 영업이익률 (%) | 5.5 | 4.5 | 5.3 | 5.4 | 5.9 |
| EBITDA | 3,068 | 2,834 | 3,111 | 3,229 | 3,516 |
| EBITDA Margin (%) | 10.2 | 9.8 | 10.5 | 10.5 | 10.9 |
| 영업외손익 | -419 | -560 | -628 | -660 | -661 |
| 관계기업손익 | 29 | 20 | 19 | 19 | 19 |
| 금융수익 | 681 | 459 | 360 | 372 | 386 |
| 금융비용 | -989 | -911 | -828 | -816 | -805 |
| 기타 | -140 | -127 | -181 | -236 | -261 |
| 법인세비용차감전순손익 | 1,246 | 732 | 940 | 1,000 | 1,242 |
| 법인세비용 | 443 | 173 | 235 | 260 | 323 |
| 계속사업순손익 | 803 | 559 | 705 | 740 | 91

In [22]:
# 엘라스틱서치를 활용한 디코딩
nid_values = [item['document_id'] for item in faiss_results]

query = {
    "query": {
        "terms": {
            "nid": nid_values
        }
    }
}

response = es.search(index=es_index_name, body=query)


In [23]:
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")

ID: mT5z2pQBLfL4KSeUiQxM, Score: 1.0
Content: {'nid': 36, 'text': '20241120에 신한투자증권에서 발행한 CJ제일제당에 관한 레포트에서 나온 내용.\n\n목표주가 370,000원(SOTP Valuation, 사업부문별 가치합산평가) 유지. 최\n근 국내 식품 매출 성장률이 둔화되는 만큼 해외 식품 매출 성장성이 중장\n기 밸류에이션 레벨을 결정할 것', 'embedding': [-0.06510689109563828, 0.00635856157168746, -0.009752815589308739, 0.03557455539703369, -0.012186789885163307, 0.055384259670972824, -0.01642419956624508, 0.007713858503848314, -0.02814530022442341, 0.025514373555779457, 0.03972245007753372, -0.00673901429399848, -0.01490862388163805, -0.0340796560049057, 0.028463805094361305, -0.027800487354397774, 0.04272972047328949, -0.09046369045972824, -0.020527256652712822, -0.03085687942802906, 0.014392450451850891, 0.016575997695326805, 0.023948775604367256, 0.04239882901310921, 0.057494133710861206, -0.005087574012577534, -0.019278666004538536, -0.0064706141129136086, 0.02649223804473877, 0.05829441919922829, 0.008768043480813503, -0.004033050034195185, 0.031013891100883484, -0.021053358912467957, 0.0475

# 메트릭 계산


In [51]:
import math
def rel_score(predictions, ground_truths, source_file, company_name):
    """
    문서의 관련성 점수를 계산합니다.
    
    Args:
        doc_id (int): 문서 ID
        qa_pairs (pd.DataFrame): 질문-답변 쌍 데이터프레임
        
    Returns:
        float: 문서의 관련성 점수
    """
    pred_scores = []

    for pred in predictions:
        score = 0
        if pred in ground_truths:
            score = 3
        elif pred in source_group[source_file]:
            score = 2
        elif pred in company_group[company_name]:
            score = 1
        else:
            score = 0
        pred_scores.append([pred, score])
    return pred_scores

def compute_ndcg(predictions, ground_truths, source_file_list, company_name_list, k=5):
    """
    각 쿼리마다 예측 결과와 ground truth(관련 문서 및 관련성 점수)를 기반으로 nDCG@k를 계산합니다.
    
    Args:
        predictions (list of list): 각 쿼리에 대해 순위가 매겨진 결과 문서 ID 리스트.
            예: [["doc1", "doc2", "doc3", ...], [...], ...]
        ground_truths (list of list): 각 쿼리의 정답 관련 문서와 관련성 점수를 담은 리스트.
            각 쿼리마다, ground truth는 (doc_id, relevance) 튜플의 리스트로 주어집니다.
            예: [[("doc2", 3), ("doc3", 1)], [("doc7", 2), ("doc9", 3)], ...]
        k (int): 상위 k개 결과만 고려 (default=5)
        
    Returns:
        float: 전체 쿼리에 대한 평균 nDCG@k
    """
    num_queries = len(predictions)
    total_ndcg = 0.0

    for pred, gt, source_file, company_name in zip(predictions, ground_truths, source_file_list, company_name_list ):
        # ground truth 관련성 정보를 빠르게 lookup하기 위해 dictionary 생성:
        # gt_dict = {doc: rel for doc, rel in gt}
        # gt_dict = {doc: rel for doc, rel in gt}

        # DCG 계산: 예측된 상위 k개 문서에 대해, ground truth에서 가져온 관련성 점수를 사용
        dcg = 0.0
        top_k = pred[:k]
        top_k_scores = rel_score(top_k, gt, source_file, company_name)
        for i, (top_k_pred, score) in enumerate(top_k_scores):
            # 해당 문서가 ground truth에 있으면 관련성 점수를, 없으면 0으로 처리
            dcg += (2 ** score - 1) / math.log2(i + 2)  # i는 0부터 시작하므로, log2(i+2)

        # IDCG 계산: 이상적인 순서(ground truth 문서들을 관련성 점수가 높은 순으로 정렬)에서의 DCG
        # ground truth 문서들을 관련성 점수가 높은 순으로 정렬하고 상위 k개만 고려합니다.
        ideal_gt = [3 for _ in range(len(gt))] + [2 for _ in range(len(source_group[source_file]) - len(gt))] + [1 for _ in range(len(company_group[company_name]) - len(source_group[source_file]))]
        idcg = 0.0
        for i, rel in enumerate(ideal_gt[:k]):
            idcg += (2 ** rel - 1) / math.log2(i + 2)

        ndcg = dcg / idcg if idcg > 0 else 0.0
        total_ndcg += ndcg

    return total_ndcg / num_queries if num_queries > 0 else 0.0



def compute_precision(predictions, ground_truths, source_file_list, company_name_list, k=5):
    """
    각 쿼리별 Precision@k를 계산합니다.
    
    Precision@k = (상위 k개 결과 중 관련 문서 수) / k
    
    Args:
        predictions (list of list): 각 쿼리에 대해 순위가 매겨진 결과 리스트
        ground_truths (list of list): 각 쿼리의 정답 문서 리스트
        k (int): 상위 k개 결과만 고려 (default=5)
        
    Returns:
        float: 전체 쿼리에 대한 평균 Precision@k
    """
    num_queries = len(predictions)
    total_precision = 0.0

    for pred, gt, source_file, company_name in zip(predictions, ground_truths, source_file_list, company_name_list):
        top_k = pred[:k]
        top_k_scores = rel_score(top_k, gt, source_file, company_name)
        num_relevant = sum(1 for pred, doc in top_k_scores if doc > 0)
        precision = num_relevant / k
        total_precision += precision

    return total_precision / num_queries if num_queries > 0 else 0.0


def compute_recall(predictions, ground_truths, source_file_list, company_name_list, k=5):
    """
    각 쿼리별 Recall@k를 계산합니다.
    
    Recall@k = (상위 k개 결과 중 관련 문서 수) / (전체 정답 문서 수)
    
    Args:
        predictions (list of list): 각 쿼리에 대해 순위가 매겨진 결과 리스트
        ground_truths (list of list): 각 쿼리의 정답 문서 리스트
        k (int): 상위 k개 결과만 고려 (default=5)
        
    Returns:
        float: 전체 쿼리에 대한 평균 Recall@k
    """
    num_queries = len(predictions)
    total_recall = 0.0

    for pred, gt, source_file, company_name in zip(predictions, ground_truths, source_file_list, company_name_list):
        top_k = pred[:k]
        top_k_scores = rel_score(top_k, gt, source_file, company_name)
        num_relevant = sum(1 for pred, doc in top_k_scores if doc > 0)
        recall = num_relevant / len(company_group[company_name]) if company_group[company_name] else 0.0
        total_recall += recall

    return total_recall / num_queries if num_queries > 0 else 0.0

def compute_mrr(predictions, ground_truths, k=5):
    """
    각 쿼리별 단일 정답(ground truth)이 포함된 예측 결과(predictions)를 기반으로 MRR를 계산합니다.
    
    Args:
        predictions (list of list): 각 쿼리에 대해 순위가 매겨진 결과 리스트 
                                    (예: [["doc1", "doc2", ...], ["doc5", "doc6", ...], ...])
        ground_truths (list): 각 쿼리의 정답 (예: ["doc2", "doc5", ...])
        k (int): 상위 k개 결과만 고려 (default=10)
        
    Returns:
        float: 전체 쿼리에 대한 평균 Reciprocal Rank (MRR)
    """
    num_queries = len(predictions)
    total_reciprocal_rank = 0.0

    for pred, gt in zip(predictions, ground_truths):
        top_k = pred[:k]
        # 정답이 top_k에 있다면 첫 등장 위치의 reciprocal rank를 계산
        try:
            for idx, doc in enumerate(top_k):
                if doc in gt:
                    total_reciprocal_rank += 1.0 / (idx + 1)
                    break
            
        except ValueError:
            # 정답이 top_k에 없으면 reciprocal rank는 0
            print("ValueError")
            total_reciprocal_rank += 0.0

    return total_reciprocal_rank / num_queries if num_queries > 0 else 0.0


def compute_hits(predictions, ground_truths, k=5):
    """
    각 쿼리별 단일 정답(ground truth)이 상위 k 결과 내에 포함되었는지를 기반으로 Hits@K를 계산합니다.
    
    Args:
        predictions (list of list): 각 쿼리에 대해 순위가 매겨진 결과 리스트 
                                    (예: [["doc1", "doc2", ...], ["doc5", "doc6", ...], ...])
        ground_truths (list): 각 쿼리의 정답 (예: ["doc2", "doc5", ...])
        k (int): 상위 k개 결과만 고려 (default=10)
        
    Returns:
        float: 전체 쿼리에 대한 평균 Hits@K (정답이 상위 k개에 포함된 쿼리의 비율)
    """
    num_queries = len(predictions)
    hit_count = 0

    for pred, gt in zip(predictions, ground_truths):
        top_k = pred[:k]
        for idx, doc in enumerate(top_k):
            if doc in gt:
                hit_count += 1
                break

    return hit_count / num_queries if num_queries > 0 else 0.0

In [52]:
# test 2. report qa_pairs 데이터셋 로드

report_qa_pairs = pd.read_csv("../data/report_qa_pairs_20250205.csv", encoding="utf-8")

In [53]:
top_k = 100

In [ ]:
predictions, ground_truths = [], []
source_file_list, company_name_list = [], []
for i, chunk in tqdm(report_qa_pairs.iterrows(), desc="test", leave=False):

    # print(chunk)
    question = chunk['question']
    document_ids = ast.literal_eval(chunk['answer_document_id'])
    company_name = chunk['company_name']
    source_file = chunk['source_file']
    
    
    ground_truths.append([document_ids])
    
    
    print("question:", question, "answer document_ids:", document_ids)
    print("document(context:", document_data[int(document_ids[0])-1].page_content)
    print("*"*50)
    
    vector = model.encode(question).tolist()  # 벡터를 리스트로 변환
    response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : vector, "size" : top_k}, timeout=15)

    faiss_results = response.json()['results']
    print(faiss_results)
    for item in faiss_results:
        # print(item)
        print(document_data[int(item['document_id'])].page_content)

    predictions.append([item['document_id']+1 for item in faiss_results])
    source_file_list.append(source_file)
    company_name_list.append(company_name)
    

In [61]:
# test 3. metric 함수 적용
metric_top_k_list = [1, 3, 5, 10, 50, 100]

for m_top_k in metric_top_k_list:
  ncdg = compute_ndcg(predictions, ground_truths, source_file_list, company_name_list, k=m_top_k)
  precision = compute_precision(predictions, ground_truths, source_file_list, company_name_list, k=m_top_k)
  recall = compute_recall(predictions, ground_truths, source_file_list, company_name_list, k=m_top_k)
  mrr = compute_mrr(predictions, ground_truths, k=m_top_k)
  hits_at_1 = compute_hits(predictions, ground_truths, k=m_top_k)
  print(f"nDCG@{m_top_k}: {ncdg:.4f}")
  print(f"Precision@{m_top_k}: {precision:.4f}")
  print(f"Recall@{m_top_k}: {recall:.4f}")
  print(f"MRR@{m_top_k}: {mrr:.4f}")
  print(f"Hits@{m_top_k}: {hits_at_1:.4f}")
  print("*"*50)


nDCG@1: 0.2443
Precision@1: 0.9966
Recall@1: 0.0029
MRR@1: 0.0000
Hits@1: 0.0000
**************************************************
nDCG@3: 0.3198
Precision@3: 0.9966
Recall@3: 0.0086
MRR@3: 0.0000
Hits@3: 0.0000
**************************************************
nDCG@5: 0.3436
Precision@5: 0.9973
Recall@5: 0.0144
MRR@5: 0.0000
Hits@5: 0.0000
**************************************************
nDCG@10: 0.3684
Precision@10: 0.9966
Recall@10: 0.0287
MRR@10: 0.0000
Hits@10: 0.0000
**************************************************
nDCG@50: 0.4579
Precision@50: 0.9822
Recall@50: 0.1411
MRR@50: 0.0000
Hits@50: 0.0000
**************************************************
nDCG@100: 0.5711
Precision@100: 0.9624
Recall@100: 0.2755
MRR@100: 0.0000
Hits@100: 0.0000
**************************************************
